In [ ]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup as bs
import xlsxwriter

In [ ]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36', 
           'x-api-source': 'pc', 'x-requested-with': 'XMLHttpRequest', 'x-shopee-language': 'zh-Hant', 
          'cookie': 'SPC_F=JhDQe7D66ZJRSo9FXG495qr4O0BloTjY; REC_T_ID=769738d6-da49-11ea-ad8e-b4969130c56c; _fbp=fb.1.1596981965618.982700371; __BWfp=c1596981977542xc7fbaee54; csrftoken=IfhLgNz00K4lnBUIa1r7D4YRLsS1EKUK; G_ENABLED_IDPS=google; SPC_CLIENTID=JhDQe7D66ZJRSo9Fgujpogfjcixryswv; SPC_IA=-1; SPC_EC=-; SPC_U=-; welcomePkgShown=true; _gcl_au=1.1.1930762698.1613442663; _med=refer; _gid=GA1.2.525977365.1617721285; AMP_TOKEN=%24NOT_FOUND; SPC_SI=mall.Lg9pybjZLOO1wPWXqHRBxBurX6lITlHi; _ga=GA1.2.659499360.1596981966; SPC_R_T_ID="0rfzVDtxrh7kycAQGSPhlEW6u4Jm8u85tNA2ZIb/RlP9ztu3aG8/L72USKNma2sZLjdbYZzzBJrTqkdCEoNP37Fc770s011lp+ZHb+WI6UI="; SPC_T_IV="PHmuMCIsyHm6jYcBsyXnyQ=="; SPC_R_T_IV="PHmuMCIsyHm6jYcBsyXnyQ=="; SPC_T_ID="0rfzVDtxrh7kycAQGSPhlEW6u4Jm8u85tNA2ZIb/RlP9ztu3aG8/L72USKNma2sZLjdbYZzzBJrTqkdCEoNP37Fc770s011lp+ZHb+WI6UI="; _ga_RPSBE3TQZZ=GS1.1.1617801050.38.1.1617801510.60', 
          'referer': 'https://shopee.tw/', 'upgrade-insecure-requests': '1', 
          'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 
          'accept-encoding': 'gzip, deflate, br', 'accept-language': 'zh-TW,zh;q=0.9,ru;q=0.8,en-US;q=0.7,en;q=0.6,ja;q=0.5'}

## Get all items

In [ ]:
main = 'https://shopee.tw/api/v4/search/search_items?by=relevancy&categoryids=10085%2C14436&keyword=arai&limit=50&newest=0&order=desc&page_type=search&skip_autocorrect=1&version=2'

In [ ]:
res = requests.get(main, headers=headers)

In [ ]:
total_items = json.loads(res.content)['adjust']['count']

In [ ]:
columns = ['商店ID', '品項名稱', '地區', '已銷售', '庫存', '評分', '價格', '折扣前價格', '折扣', '最高價', '折扣前最高價', '最低價', '折扣前最低價', '已瀏覽次數']
df = pd.DataFrame(columns=columns)

for i in range(0, int(total_items / 50)):
    n = i * 50
    url = 'https://shopee.tw/api/v4/search/search_items?by=relevancy&categoryids=10085%2C14436&keyword=arai&limit=50&newest=' + str(n) + '&order=desc&page_type=search&skip_autocorrect=1&version=2'
    res = requests.get(url, headers=headers)
    data = json.loads(res.content)
    for i in data['items']:
        discount = i['item_basic']['discount']
        sold_item = i['item_basic']['historical_sold']
        rating = i['item_basic']['item_rating']['rating_star']
        name = i['item_basic']['name']
        price = i['item_basic']['price']
        price_before_discount = i['item_basic']['price_before_discount']
        price_max = i['item_basic']['price_max']
        price_max_before_discount = i['item_basic']['price_max_before_discount']
        price_min = i['item_basic']['price_min']
        price_min_before_discount = i['item_basic']['price_min_before_discount']
        location = i['item_basic']['shop_location']
        shopid = i['item_basic']['shopid']
        stock = i['item_basic']['stock']
        views = i['item_basic']['view_count']
        df = df.append({'商店ID': shopid, '品項名稱': name, '地區': location, '已銷售': sold_item,
                        '庫存': stock, '評分': rating, '價格': price, '折扣前價格': price_before_discount,
                        '折扣': discount, '最高價': price_max, '折扣前最高價': price_max_before_discount,
                        '最低價': price_min, '折扣前最低價': price_min_before_discount, '已瀏覽次數': views}, 
                       ignore_index=True)

## 數據處理

In [ ]:
df['價格'] = df['價格'] / 100000
df['折扣前價格'] = df['折扣前價格'] / 100000
df['最高價'] = df['最高價'] / 100000
df['折扣前最高價'] = df['折扣前最高價'] / 100000
df['最低價'] = df['最低價'] / 100000
df['折扣前最低價'] = df['折扣前最低價'] / 100000

In [ ]:
for i in ['已銷售', '庫存', '價格', '折扣前價格', '最高價',
       '折扣前最高價', '最低價', '折扣前最低價', '已瀏覽次數']:
    df[i] = df[i].astype('int')

In [ ]:
test = df.copy()

In [ ]:
test['平均價格'] = (test['最高價'] + test['最低價']) / 2

In [ ]:
test['平均價格'] = test['平均價格'].astype('int')

In [ ]:
test['營業額'] = test['平均價格'] * test['已銷售']
test['庫存金額'] = test['平均價格'] * test['庫存']

In [ ]:
test.to_excel('shopee_arai.xlsx', index=False, engine='xlsxwriter')